# AEMO 2024 ISP Progressive Change - Network Analysis

This notebook performs comprehensive analysis and visualization of the solved AEMO 2024 Integrated System Plan (ISP) Progressive Change scenario.

## Analysis Overview

This notebook will analyze:
1. **Energy Balance** - Supply and demand patterns across NEM regions
2. **Generation Capacity** - Optimal capacity by technology
3. **Capacity Factors** - Generator utilization rates
4. **Economic Analysis** - Capital and operational costs
5. **Comprehensive Dashboard** - Multi-metric overview

All visualizations use **Tableau 10 color palette** for professional, publication-ready figures.

## AEMO Context

The AEMO 2024 ISP Progressive Change scenario models Australia's National Electricity Market (NEM) transition, including:
- High renewable energy penetration
- Coal retirement schedules
- Interstate transmission expansion
- Regional demand patterns (QLD, NSW, VIC, SA, TAS)

---

## Setup and Configuration

In [ ]:
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import pypsa

# Add src to path
sys.path.insert(0, str(Path.cwd().parent.parent))

from plexos_to_pypsa_converter.analysis import NetworkAnalyzer
from plexos_to_pypsa_converter.analysis.styles import apply_default_style

# Apply default styling
apply_default_style()

# Configure matplotlib for inline display
%matplotlib inline
plt.rcParams["figure.dpi"] = 100
plt.rcParams["savefig.dpi"] = 150
plt.rcParams["figure.figsize"] = (10, 6)

print("✓ Imports and configuration complete")

## Load Solved Network

Load the network file created by `aemo_solve.ipynb`:

In [ ]:
# Define path to solved network
network_file = (
    Path.cwd().parent
    / "results"
    / "aemo-2024-isp-progressive-change"
    / "solved_network_2025.nc"
)

if not network_file.exists():
    raise FileNotFoundError(
        f"Network file not found: {network_file}\n"
        "Please run aemo_solve.ipynb first to generate the solved network."
    )

# Load network
print(f"Loading network from: {network_file.name}")
network = pypsa.Network()
network.import_from_netcdf(str(network_file))

print("✓ Network loaded successfully")
print(f"  File size: {network_file.stat().st_size / 1024 / 1024:.2f} MB")

## Initialize Network Analyzer

Create the `NetworkAnalyzer` with **Tableau 10 color palette** for consistent, professional visualizations:

In [ ]:
# Initialize analyzer with Tableau colors
analyzer = NetworkAnalyzer(
    network, exclude_slack_default=True, color_palette="tableau10"
)

print("✓ NetworkAnalyzer initialized with Tableau 10 color palette")
print(f"  Spatial resolution: {analyzer.spatial_resolution}")
print(f"  Multi-period: {analyzer.multi_period}")
print(f"  Slack generators: {len(analyzer.slack_generators)}")

---

# Network Overview

## Network Statistics

Display key statistics about the AEMO network structure:

In [ ]:
# Get network info
info = analyzer.info()

print("=" * 70)
print(" AEMO 2024 ISP Progressive Change - Network Statistics")
print("=" * 70)
print()
print(f"Buses:              {info['buses']:>6}")
print(f"Generators:         {info['generators']:>6}")
print(f"Loads:              {info['loads']:>6}")
print(f"Links:              {info['links']:>6}")
print(f"Storage Units:      {info['storage_units']:>6}")
print(f"Stores:             {info['stores']:>6}")
print()
print(f"Snapshots:          {info['snapshots']:>6}")

# Display investment periods
inv_periods = info["investment_periods"]
if inv_periods > 0:
    print(f"Investment Periods: {inv_periods:>6} (optimized for 2025)")
else:
    print(f"Investment Periods: {'N/A':>6}")

print(f"Carriers:           {info['carriers']:>6}")
print(f"Bus Carriers:       {len(info['bus_carriers']):>6}")
print()
print(f"Spatial Resolution: {info['spatial_resolution']}")
print(f"Multi-period:       {info['multi_period']}")
print()
print(f"Has Time Series:    {info['has_time_series']}")
print(f"Has Storage:        {info['has_storage']}")
print(f"Has Stores:         {info['has_stores']}")
print(f"Has Links:          {info['has_links']}")
print()
print("=" * 70)

## NEM Regions

Display the buses representing Australian NEM regions:

In [ ]:
# Display NEM regions
print(f"NEM Regions (Buses): {len(network.buses)}\n")
for i, bus in enumerate(network.buses.index, 1):
    print(f"  {i}. {bus}")

## Generator Carriers

List all energy carriers (technologies) in the model:

In [ ]:
# Display carriers
carriers = sorted(network.generators.carrier.unique())
print(f"Total unique carriers: {len(carriers)}\n")

# Create a formatted list
for i, carrier in enumerate(carriers, 1):
    if i % 3 == 1:
        print(f"  {i:2d}. {carrier:25s}", end="")
    elif i % 3 == 0:
        print(f"  {i:2d}. {carrier}")
    else:
        print(f"  {i:2d}. {carrier:25s}", end="")
if len(carriers) % 3 != 0:
    print()

---

# Energy Balance Analysis

## Understanding Energy Balance

Energy balance shows the **supply** (positive) and **withdrawal** (negative) of energy by carrier:
- **Supply**: Energy generated or supplied to the NEM (generators, imports)
- **Withdrawal**: Energy consumed or withdrawn from the NEM (loads, exports, losses)

This analysis helps us understand:
- Which technologies provide the most energy to the NEM
- How energy is consumed across Australian regions
- The overall energy mix in the Progressive Change scenario

## Figure 1: Total Energy Balance by Carrier

In [ ]:
# Create energy balance plot
fig, ax = plt.subplots(figsize=(12, 8))
analyzer.plot_energy_balance_totals(ax=ax)
plt.tight_layout()
plt.show()

print(
    "Figure 1: Total energy balance showing supply (right) and withdrawal (left) by carrier."
)
print("Colors: Tableau 10 palette")

### Energy Balance Metrics

Extract key energy metrics:

In [ ]:
# Get supply and withdrawal data
supply = analyzer.get_supply()
withdrawal = analyzer.get_withdrawal()

print("Top 5 Energy Suppliers:")
print("=" * 50)
for carrier, value in supply.head(5).items():
    carrier_str = str(carrier) if not isinstance(carrier, str) else carrier
    print(f"  {carrier_str:30s} {value:>15,.0f} MWh")

print("\nTop 5 Energy Withdrawals:")
print("=" * 50)
for carrier, value in withdrawal.head(5).items():
    carrier_str = str(carrier) if not isinstance(carrier, str) else carrier
    print(f"  {carrier_str:30s} {value:>15,.0f} MWh")

print(f"\nTotal Supply:    {supply.sum():>15,.0f} MWh")
print(f"Total Withdrawal: {abs(withdrawal.sum()):>15,.0f} MWh")

---

# Generation Capacity Analysis

## Understanding Capacity

**Optimal Capacity** represents the installed generation capacity (in MW) for each technology in 2025. This shows:
- The mix of generation technologies in the NEM
- Relative size of each technology's capacity
- Investment decisions for the Progressive Change scenario

## Figure 2: Optimal Capacity by Carrier

In [ ]:
# Create capacity overview plot
fig, ax = plt.subplots(figsize=(12, 7))
analyzer.plot_capacity_overview(capacity_type="optimal", groupby="carrier", ax=ax)
plt.tight_layout()
plt.show()

print("Figure 2: Optimal generation capacity by carrier (MW).")
print("Shows the installed capacity for each technology type in 2025.")

### Capacity Metrics

In [ ]:
# Get capacity data
capacity = analyzer.get_optimal_capacity()
total_capacity = capacity.sum()

print("Top 10 Technologies by Capacity:")
print("=" * 70)
print(f"{'Carrier':<30s} {'Capacity (MW)':>15s} {'Share':>10s}")
print("=" * 70)

for carrier, cap in capacity.head(10).items():
    carrier_str = str(carrier) if not isinstance(carrier, str) else carrier
    share = cap / total_capacity * 100
    print(f"{carrier_str:<30s} {cap:>15,.0f} {share:>9.1f}%")

print("=" * 70)
print(f"{'TOTAL':<30s} {total_capacity:>15,.0f} {100.0:>9.1f}%")
print("=" * 70)

## Capacity Factors

**Capacity Factor** measures how much a generator is actually used relative to its maximum capacity:
- **1.0 (100%)**: Generator runs at full capacity continuously
- **0.5 (50%)**: Generator operates at half capacity on average
- **Low values**: Variable renewables (solar, wind) or peaking plants
- **High values**: Baseload plants (coal, nuclear) or must-run facilities

## Figure 3: Capacity Factors by Carrier

In [ ]:
# Create capacity factors plot
fig, ax = plt.subplots(figsize=(12, 7))
analyzer.plot_capacity_factors(groupby="carrier", ax=ax)
plt.tight_layout()
plt.show()

print("Figure 3: Capacity factors by carrier (utilization rate, 0-1).")
print("Higher values indicate more consistent operation.")

### Capacity Factor Analysis

In [ ]:
# Get capacity factor data
cf = analyzer.get_capacity_factors()

print("Capacity Factors by Technology:")
print("=" * 60)
print(f"{'Carrier':<30s} {'Capacity Factor':>15s} {'Utilization':>10s}")
print("=" * 60)

for carrier, factor in cf.head(15).items():
    carrier_str = str(carrier) if not isinstance(carrier, str) else carrier
    print(f"{carrier_str:<30s} {factor:>15.3f} {factor * 100:>9.1f}%")

print("=" * 60)

# Summary statistics
print(f"\nMean Capacity Factor: {cf.mean():.3f} ({cf.mean() * 100:.1f}%)")
print(f"Median Capacity Factor: {cf.median():.3f} ({cf.median() * 100:.1f}%)")

---

# Economic Analysis

## Understanding Costs

The model tracks two types of costs:
- **CAPEX (Capital Expenditure)**: Upfront investment costs for building capacity
- **OPEX (Operational Expenditure)**: Ongoing costs for fuel, maintenance, and operation

This analysis shows:
- Which technologies have high capital vs. operating costs
- The economic trade-offs between technologies in the NEM
- Total system costs for 2025

## Figure 4: Cost Breakdown (CAPEX + OPEX)

In [ ]:
# Create cost breakdown plot
fig, ax = plt.subplots(figsize=(12, 7))
analyzer.plot_cost_breakdown(groupby="carrier", ax=ax)
plt.tight_layout()
plt.show()

print("Figure 4: Cost breakdown showing CAPEX (blue) and OPEX (red) by carrier.")
print("Total height represents total cost per technology.")

## Figure 5: CAPEX vs OPEX Comparison

In [ ]:
# Create cost comparison plot
fig, axes = analyzer.plot_cost_comparison(groupby="carrier", figsize=(14, 6))
plt.tight_layout()
plt.show()

print("Figure 5: Side-by-side comparison of CAPEX and OPEX by carrier.")
print("Left: Capital costs | Right: Operating costs")

### Cost Metrics

In [ ]:
# Get cost data
costs = analyzer.get_costs(cost_type="total")
capex = costs["capex"]
opex = costs["opex"]
total = costs["total"]

print("Top 10 Technologies by Total Cost:")
print("=" * 90)
print(
    f"{'Carrier':<25s} {'CAPEX ($)':>15s} {'OPEX ($)':>15s} {'Total ($)':>15s} {'Share':>8s}"
)
print("=" * 90)

for carrier in total.head(10).index:
    carrier_str = str(carrier) if not isinstance(carrier, str) else carrier
    cap_cost = capex.get(carrier, 0)
    op_cost = opex.get(carrier, 0)
    tot_cost = total[carrier]
    share = tot_cost / total.sum() * 100
    print(
        f"{carrier_str:<25s} {cap_cost:>15,.0f} {op_cost:>15,.0f} {tot_cost:>15,.0f} {share:>7.1f}%"
    )

print("=" * 90)
total_capex = capex.sum()
total_opex = opex.sum()
total_cost = total.sum()
print(
    f"{'TOTAL':<25s} {total_capex:>15,.0f} {total_opex:>15,.0f} {total_cost:>15,.0f} {100.0:>7.1f}%"
)
print("=" * 90)

print("\nCost Breakdown:")
print(f"  CAPEX: ${total_capex:,.0f} ({total_capex / total_cost * 100:.1f}%)")
print(f"  OPEX:  ${total_opex:,.0f} ({total_opex / total_cost * 100:.1f}%)")

---

# Comprehensive Dashboard

## Multi-Metric Overview

This dashboard combines the key metrics from all previous analyses into a single view:
- **Top Left**: Energy balance totals
- **Top Right**: Optimal capacity by carrier
- **Bottom Left**: Cost breakdown (CAPEX + OPEX)
- **Bottom Right**: Capacity factors

All plots use consistent **Tableau 10 colors** for easy comparison across metrics.

## Figure 6: Network Analysis Dashboard

In [ ]:
# Create comprehensive dashboard
fig = analyzer.create_dashboard(figsize=(16, 12))
plt.tight_layout()
plt.show()

print("Figure 6: Comprehensive dashboard with all key metrics.")
print("All plots use consistent Tableau 10 color palette.")

---

# Key Findings & Summary

## Summary Statistics Table

In [ ]:
# Create summary statistics DataFrame
summary_data = {
    "Metric": [
        "Total Capacity",
        "Total Energy Supply",
        "Total Energy Withdrawal",
        "Mean Capacity Factor",
        "Total CAPEX",
        "Total OPEX",
        "Total System Cost",
        "Number of Generators",
        "Number of Carriers",
        "Number of Buses",
        "Optimization Year",
    ],
    "Value": [
        f"{capacity.sum():,.0f} MW",
        f"{supply.sum():,.0f} MWh",
        f"{abs(withdrawal.sum()):,.0f} MWh",
        f"{cf.mean():.3f} ({cf.mean() * 100:.1f}%)",
        f"${capex.sum():,.0f}",
        f"${opex.sum():,.0f}",
        f"${total.sum():,.0f}",
        f"{len(network.generators)}",
        f"{len(carriers)}",
        f"{len(network.buses)}",
        "2025",
    ],
}

summary_df = pd.DataFrame(summary_data)
print("=" * 70)
print(" AEMO 2024 ISP Progressive Change - Summary (2025)")
print("=" * 70)
print()
print(summary_df.to_string(index=False))
print()
print("=" * 70)

## Key Insights - AEMO Progressive Change Scenario

Based on the analysis above, we can observe for the 2025 investment period:

### Energy Mix in the NEM
- The Progressive Change scenario shows the distribution of energy generation across different technologies
- Supply and withdrawal patterns reveal the primary energy flows across Australian regions
- Transition from coal to renewable energy sources

### Capacity Deployment
- Optimal capacity shows investment decisions for 2025
- The capacity mix reflects the balance between:
  - Coal retirement schedules
  - Renewable energy expansion (wind, solar)
  - Firming capacity (batteries, gas)

### Technology Utilization
- Capacity factors reveal how consistently different technologies operate in the NEM
- Variable renewables (wind, solar) show lower capacity factors due to intermittency
- Baseload and firming plants show higher capacity factors

### Economic Trade-offs
- The cost analysis shows the balance between:
  - Capital-intensive renewables (high CAPEX, low OPEX)
  - Fuel-intensive thermal generation (lower CAPEX, higher OPEX)
- Progressive Change scenario investment priorities

### Regional Considerations
- Interstate transmission flows between NEM regions (QLD, NSW, VIC, SA, TAS)
- Regional generation and demand patterns
- Interconnector utilization

---

## Export Figures

Optionally save all figures to files for reports or presentations:

In [ ]:
# Uncomment to save figures
# output_dir = Path("figures")
# output_dir.mkdir(exist_ok=True)

# print("Saving figures...")
# analyzer.plot_energy_balance_totals(save_path=output_dir / "aemo_energy_balance.png", dpi=300)
# analyzer.plot_capacity_overview(save_path=output_dir / "aemo_capacity.png", dpi=300)
# analyzer.plot_capacity_factors(save_path=output_dir / "aemo_capacity_factors.png", dpi=300)
# analyzer.plot_cost_breakdown(save_path=output_dir / "aemo_costs.png", dpi=300)
# analyzer.create_dashboard(save_path=output_dir / "aemo_dashboard.png", dpi=300)
# print(f"✓ Figures saved to {output_dir}/")

print("To save figures, uncomment the code above and run this cell.")

---

## Analysis Complete

✓ All visualizations generated with **Tableau 10 color palette**

✓ Comprehensive analysis of AEMO 2024 ISP Progressive Change scenario (2025)

✓ Key findings summarized above

### Next Steps

You can:
1. Modify the analysis by changing parameters in the cells above
2. Export figures for reports (uncomment export section)
3. Compare with other investment periods (2030, 2035, 2040, 2050)
4. Analyze specific NEM regions or carriers in detail
5. Compare with other AEMO scenarios (Green Energy Exports, Step Change)

---